In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm_notebook as tqdm
import csv
import pickle

In [2]:
X_train, Y_train = pickle.load(open('filtered_tfidf_50K.pkl', 'rb'))
print(X_train.shape, Y_train.shape)

(1903408, 50000) (1903408,)


In [3]:
all_classes = Y_train.unique()
len(all_classes)

2759

In [ ]:
from sklearn.naive_bayes import GaussianNB
pagr = GaussianNB()
start = 0
minibatch_size = 30000
for i in tqdm(range(0, (int)(X_train.shape[0]/minibatch_size))):
    
    train_x = X_train[start:(start+minibatch_size), :].todense()
    train_y = Y_train.iloc[start:(start+minibatch_size)]
    pagr.partial_fit(train_x, train_y, classes=all_classes)
    #_, test_x, _, test_y = train_test_split(X_test,Y_test, test_size=0.1)
    #print(f"Accuracy = {mnb.score(test_x, test_y)}")
    start+=minibatch_size

In [5]:
test = pd.read_csv('test.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [7]:
test_final = pickle.load(open('test_tfidf_50K','rb'))
test_final.shape

(110775, 50000)

In [8]:
pred = pagr.predict(test_final) 
print(pred.shape)

(110775,)


In [9]:
submission = pd.DataFrame(zip(test['PRODUCT_ID'], pred), columns = ['PRODUCT_ID', 'BROWSE_NODE_ID'])
submission.to_csv('submission8.csv', index = False)

In [6]:
import collections
class_avg = collections.defaultdict(lambda: np.zeros((1, 50000))) # maps: class label -> [avg vector, no. of data points]
class_count = collections.defaultdict(int)
print(type(X_train))
print(type(Y_train))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.series.Series'>


In [7]:
from scipy.sparse import csr_matrix

In [8]:
batch_size = 20000
start = 0
for i in range(0, (int)(X_train.shape[0]/batch_size)):
    x_sample = X_train[start:(start+batch_size)]
    y_sample = Y_train[start:(start+batch_size)]

    # convert to dense matrix
    x_sample = x_sample.todense()

    for j,label in enumerate(y_sample):
        class_avg[label] = np.add(class_avg[label], x_sample[j])
        class_count[label] += 1
    print(f"Populated dict for batch {i}")
    start+=batch_size

Populated dict for batch 0
Populated dict for batch 1
Populated dict for batch 2
Populated dict for batch 3
Populated dict for batch 4
Populated dict for batch 5
Populated dict for batch 6
Populated dict for batch 7
Populated dict for batch 8
Populated dict for batch 9
Populated dict for batch 10
Populated dict for batch 11
Populated dict for batch 12
Populated dict for batch 13
Populated dict for batch 14
Populated dict for batch 15
Populated dict for batch 16
Populated dict for batch 17
Populated dict for batch 18
Populated dict for batch 19
Populated dict for batch 20
Populated dict for batch 21
Populated dict for batch 22
Populated dict for batch 23
Populated dict for batch 24
Populated dict for batch 25
Populated dict for batch 26
Populated dict for batch 27
Populated dict for batch 28
Populated dict for batch 29
Populated dict for batch 30
Populated dict for batch 31
Populated dict for batch 32
Populated dict for batch 33
Populated dict for batch 34
Populated dict for batch 35
Po

In [9]:
for label, vec in class_avg.items():
    class_avg[label] = class_avg[label]/class_count[label]

In [11]:
array_seq = []
for label, vec in class_avg.items():
    array_seq.append(vec)

In [12]:
avg_arrays = np.stack(array_seq, axis = 0)
avg_arrays.shape

(2759, 50000)

In [13]:
X_test = pickle.load(open('test_tfidf_50K','rb'))
X_test.shape

(110775, 50000)

In [15]:
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
%time 
sim_matrix = cosine_similarity(X_test, avg_arrays)

Wall time: 0 ns


In [16]:
sim_matrix.shape

(110775, 2759)

In [17]:
pred_idx = np.argmax(sim_matrix, axis = 1)

In [18]:
idx_label = dict()
for i, label in enumerate(class_avg):
    idx_label[i] = label

In [19]:
pred = [idx_label[i] for i in pred_idx]

In [20]:
len(pred)

110775

In [21]:
test = pd.read_csv('test.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [22]:
submission = pd.DataFrame(zip(test['PRODUCT_ID'], pred), columns = ['PRODUCT_ID', 'BROWSE_NODE_ID'])
submission.to_csv('submission12.csv', index = False)